In [70]:
import pandas as pd
import numpy as np
from groq import Groq
import os
import pinecone
import re
import sys
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_pinecone import PineconeVectorStore
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

from IPython.display import display, HTML

from mistralai import Mistral



In [ ]:

api_key = 'add-your-api-key'
model = "mistral-large-2407"

client = Mistral(api_key=api_key)


In [ ]:
# #Gender
df = pd.read_csv('/Data/target_gender_ads.csv', index_col=False) #gender


#Age
#df = pd.read_csv('/Data/target_age_ads.csv', index_col=False) #age

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.reset_index(drop=True)
print(df.columns, df.shape)

In [73]:
assistant_replies = []

In [ ]:
import time

def chat_completion(client, model, user_question):
    try:
        chat_completion = client.chat.complete(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": user_question,
                },
            ]
        )
        
        response = chat_completion.choices[0].message.content
        return response

    except Exception as e:
        # Check if the exception is a rate limit error
        if "Rate limit exceeded" in str(e) or getattr(e, 'status_code', None) == 429:
            print("Rate limit exceeded. Waiting for 10 seconds...")
            time.sleep(10)
            return chat_completion(client, model, user_question)  # Retry after waiting
        else:
            # If it's a different error, print it and raise again
            print(f"An error occurred: {e}")
            raise



In [75]:
def remove_links(text):
    # Define a regular expression pattern to match URLs
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # Remove URLs from the text
    text_without_links = re.sub(url_pattern, '', text)

    return text_without_links

def remove_line_breaks_and_links(text):
    # Remove line breaks
    text_without_line_breaks = re.sub(r'\n{2,}', '\n', text)
    remove_leading_indent = re.sub(r'\n\s+', '\n', text_without_line_breaks)  # Remove leading indentation
    
    # Remove links
    text_without_links = remove_links(remove_leading_indent)

    return text_without_links

In [ ]:
for index, row in df.iterrows():
    text = row['ad_creative_body']
    cleaned_text = remove_line_breaks_and_links(text).strip()
    #age group
    #user_message = "Ad: "+cleaned_text+"\n Report who can be targeted based on the following four age groups: Young adults (18-24), Early working age (25-44), Late working age (45-64), and Senior citizens (65+) by the provided ad. Choose the most probable option from the age groups. Don't generate neutral or multiple age group answers. Provide a brief explanation with the reasoning in 20 words.  \n Age group: ? \n Explanation: ?"
    
    #gender
    user_message = "Ad: "+cleaned_text+"\n Report who can be targeted based on gender: male vs female by the provided ad. Choose the most probable option from the gender (either male or female). Don't generate neutral or both gender answers. Provide a brief explanation with reasoning in 20 words. \n Gender: ? \n Explanation: ?"
   
    
    assistant_reply = chat_completion(client, model, user_message)
    
    #assistant_reply = make_api_request(user_question)
    #print("assistant_reply", assistant_reply)
    #sys.exit()
    # Append the assistant's reply to the list
    assistant_replies.append(assistant_reply)
    
# Add the assistant's replies to the DataFrame
df['mistral_reply'] = assistant_replies

In [ ]:
# df.to_csv('/Data/age_mistral.csv')
df.to_csv('/Data/gender_mistral.csv')